In [1]:
import numpy as np
import opensimplex
import random
from time import sleep
import scipy.linalg as sla
import numba 
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule

In [2]:
a = np.random.randn(4,4)
a = a.astype(np.float32)
a_gpu = cuda.mem_alloc(a.nbytes)
cuda.memcpy_htod(a_gpu, a)
mod = SourceModule("""
  __global__ void doublify(float *a)
  {
    int idx = threadIdx.x + threadIdx.y*4;
    a[idx] *= 2;
  }
  """)
func = mod.get_function("doublify")
func(a_gpu, block=(4,4,1))
a_doubled = np.empty_like(a)
cuda.memcpy_dtoh(a_doubled, a_gpu)
print(a_doubled)
print(a)

[[-0.25400722  0.64964914  0.9382808  -0.01188704]
 [ 1.20344     1.6254195   0.9496643  -4.150607  ]
 [ 2.705023   -1.5061293  -1.00944     1.4249817 ]
 [-1.9821739   1.3959916  -1.0084743   0.35770354]]
[[-0.12700361  0.32482457  0.4691404  -0.00594352]
 [ 0.60172     0.81270975  0.47483215 -2.0753036 ]
 [ 1.3525115  -0.75306463 -0.50472     0.71249086]
 [-0.99108696  0.6979958  -0.5042372   0.17885177]]


In [10]:
code = open("cudaKernels/createInitialPopulation.cu", "r").read()

In [11]:
# code = """
#     #include <curand_kernel.h>

#     const int nstates = 1024;
#     __device__ curandState_t* states[nstates];

#     __global__ void initkernel(int seed)
#     {
#         int tidx = threadIdx.x + blockIdx.x * blockDim.x;

#         if (tidx < nstates) {
#             curandState_t* s = new curandState_t;
#             if (s != 0) {
#                 curand_init(seed, tidx, 0, s);
#             }

#             states[tidx] = s;
#         }
#     }

#     __global__ void randfillkernel(float *values, int N)
#     {
#         int tidx = threadIdx.x + blockIdx.x * blockDim.x;

#         if (tidx < nstates) {
#             curandState_t s = *states[tidx];
#             for(int i=tidx; i < N; i += blockDim.x * gridDim.x) {
#                 values[i] = curand_uniform(&s);
#             }
#             *states[tidx] = s;
#         }
#     }
# """
print(code)


/***
Assumptopn
    Invoked via (number of active grids,1,1) (bx,by,xz)
    where bx * by * xz = number of active threads 
***/
#define tIdx threadIdx.x
#define tIdy threadIdx.y
#define tIdz threadIdx.z

#define bDmx blockDim.x
#define bDmy blockDim.y
#define bDmz blockDim.z

#define bIdx blockIdx.x
#define bIdy blockIdx.y
#define bIdz blockIdx.z

#define gDmx gridDim.x
#define gDmy gridDim.y
#define gDmz gridDim.z

#define threadsPerBlock bDmx*bDmy*bDmz

#define WARP_SIZE 32

#include <curand_kernel.h>


__device__ __forceinline__ int row_major_4D(int w, int x, int y, int z, int W, int X,int Y, int Z){
    return z + Z * ( y + Y * ( x + X * w ) );
}

__device__ __forceinline__ int col_major_4D(int w, int x, int y, int z, int W, int X,int Y, int Z){
    return w + W *( x + X * (y + Y * z) );
}

__device__ __forceinline__ int col_major_3D(int x, int y, int z, int X,int Y, int Z){
    return x + X * (y + Y * z) ;
}

extern "C"{
__global__ void create_initial_population(int *population,co

In [12]:
mod1 = SourceModule(code, no_extern_c=True)

In [ ]:
import pycuda.driver as drv
drv.Device(0).get_attributes()